In [1]:
import numpy as np
import random
import statsmodels.api as sm
import os
from statsmodels.sandbox.stats.multicomp import multipletests
import pybedtools
from pyBedGraph import BedGraph
from pybedtools import BedTool
from sys import argv
import multiprocessing
import defs_v2

import time

Python(53961) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53962) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53963) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53964) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53965) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53966) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53967) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53968) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53969) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53970) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(53971) Malloc

In [3]:
# FDR: 0.1
# sample_size = 5000
# For 'chr1':
#   # Pass
#   # Fail
#   # Runtime/memory

# Read in samples from bed/bedgraph file
def read_gems(directory, file_name, chr_name):
    """
    Read a GEM file of the form "PlinePgem".
    Args:
       directory (str): directory of the file location (ex: '/Users/kimm/')
       file_name (str): name of the file (ex: 'SHG0008H.Fragnum_PlinePgem')
    Returns:
       in_gems (list): tab-separated lists of lists
    """
    subset_gems = {}
    cnt_all = 0
    cnt_subset = 0
    with open(directory + file_name) as f:
        for line in f:
            tmp = line.strip().split("\t")
            if tmp[0] == chr_name:
                subset_gems.setdefault(tmp[3].split("/")[0], []).append(tmp)
                cnt_subset += 1
            cnt_all += 1
        #in_gems = [line.strip().split("\t") for line in f]
    gem_span = int(subset_gems[list(subset_gems.keys())[0]][0][2]) - int(subset_gems[list(subset_gems.keys())[0]][0][1])
    return subset_gems, cnt_all, cnt_subset, gem_span

7508508
648188


In [4]:
# Read the hg38.chrom.sizes file into a dictionary
def read_chroms(directory, genome_name):
    """
    Read a tab-delimited text file with list of chromosomes and their sizes.
    Args:
       directory (str): directory of the file location (ex: '/Users/kimm/')
       genome_name (str): name of reference genome (ex: 'dm3', 'mm10', 'hg38', 'hg19', etc.)
                        Note: must have a text file <genome_name>.chrom.sizes in the directory.
    Returns:
       chrom_dict (dictionary): tab-separated lists of lists
    """
    chrom_dict = {}
    with open(directory + genome_name + '.chrom.sizes') as f:
        for line in f:
            tmp_list = line.strip().split("\t")
            chrom_dict[tmp_list[0]] = int(tmp_list[1])
    return chrom_dict


# Determine random start locations for samples to compare to observed sample
# Gem span should be the same for all single-end fragments
def random_loc_se(chrom_size, gem_span, chrom, sample_size, cov):
    """
    Randomly locate GEM with same span in a chromosome sample_size times.
    Args:
       chrom_size (int): size of a given chromosome (ex: 23011544 if 'chr2L')
       gem_span (int): GEM from start of first fragment to end of last fragment (ex: 5000)
       sample_size (int): number of random locations to sample
    Returns:
       startpos (array): random integers array of length 'sample_size'
    """
    np.random.seed(12345)
    exp_enrich_lst = []
    i = 0
    while i < sample_size:
        startpos = np.random.randint(low = 0, high = chrom_size - gem_span - 1)
        current_frag = [chrom, startpos, startpos + gem_span]
        exp_enrich = cov.stats(stat = 'max', intervals = [current_frag])[0]
        #outfile.write("CurrentIndex: " + str(i) + "\t")
        #outfile.write("Exp Enrichment: " + str(exp_enrich) + "\n")
        #outfile.write("Current Pseudo: " + str(pseudo) + "\n")
        if int(exp_enrich) == -1:
            i -= 1
        else:
            exp_enrich_lst.append(exp_enrich)
        #outfile.write("CurrentIndex SameCycle: " + str(i) + "\t")
        #outfile.write("Exp Enrichment SameCycle: " + str(exp_enrich) + "\n")
        #outfile.write("Current Pseudo SameCycle: " + str(pseudo) + "\n")
        i += 1
    return exp_enrich_lst
    
'''
def random_loc_pe(subset_gems, chrom_size, chrom, sample_size, cov):
    print("In old random_loc\n")
    np.random.seed(12345)
    exp_enrich_lst = []
    for value in subset_gems.values():
        i = 0
        start_idx = int(value[0][1])
        total_span = int(value[-1][2]) - start_idx
        one_samp_exp_enrich = []
        while i < sample_size:
            startpos = np.random.randint(low = 0, high = chrom_size - total_span - 1)
            current_frag = []
            for elem in value:
                current_frag.append([chrom, int(startpos + int(elem[1])-start_idx), int(startpos + int(elem[2])-start_idx)])
            exp_lst = list(cov.stats(stat = 'max', intervals = current_frag))
            exp_enrich = sum(exp_lst)/len(exp_lst)
            if int(exp_enrich) == -1:
                i -= 1
            else:
                #exp_enrich_dct.setdefault(key, []).append(exp_enrich)
                one_samp_exp_enrich.append(exp_enrich)
            i += 1
        exp_enrich_lst.append(one_samp_exp_enrich)
    return exp_enrich_lst
'''
'''
def random_loc_pe(subset_gems, chrom_size, chrom, sample_size, cov, out):
    print("In new random_loc_pe")
    out.write("In new random_loc_pe\n")
    np.random.seed(12345)
    exp_enrich_lst = []
    tot_pseudo_frag = []
    for value in subset_gems.values():
        start_idx = int(value[0][1])
        total_span = int(value[-1][2]) - start_idx
        startpos = np.random.randint(low = 0, high = chrom_size - total_span - 1, size = sample_size)
        for pos in startpos:
            pseudo_frag = []
            for elem in value:
                pseudo_frag.append([chrom, pos + int(elem[1]) - start_idx, pos + int(elem[2]) - start_idx])
            tot_pseudo_frag.append(pseudo_frag)
            #out.write(str(tot_pseudo_frag))
    print("Pseudos created")
    out.write("Pseudos created\n")
    one_d_frags = sum(tot_pseudo_frag, [])
    exp_lst = list(cov.stats(stat = 'max', intervals = one_d_frags))
    out.write("Enrichments found\n")
    exp_lst_arr = []
    # Rearrange exp_lst to keep maximums for the same value grouped together
    for validx, value in enumerate(subset_gems.values()):
        avg_len = len(value)
        exp_lst_arr.append([exp_lst[i:i + avg_len] for i in range(validx*sample_size, validx*sample_size + sample_size, avg_len)])
    out.write("Enrichments Arranged: " + str(exp_lst_arr[0]) + "\n")
    ## All occurences of -1 in exp_lst:
    faulty_idxs = [index for index, enr_val in enumerate(exp_lst_arr) if (-1 in enr_val)]
    for badidx in faulty_idxs:
        out.write("Bad: " + str(exp_lst_arr[badidx]) + "\t")
        while -1 in exp_lst_arr[badidx]:
            badfrag = tot_pseudo_frag[badidx]
            start_idx = int(badfrag[0][1])
            total_span = int(badfrag[-1][2]) - start_idx
            startpos2 = np.random.randint(low = 0, high = chrom_size - total_span - 1) - start_idx
            better_frag = []
            for elem in tot_pseudo_frag[badfrag]:
                better_frag.append([chrom, startpos2 + int(elem[1]), startpos2 + int(elem[2])])
            exp_lst_arr[badidx] = list(cov.stats(stat = 'max', intervals = better_frag))
        out.write("Better: " + str(exp_lst_arr[badidx]) + "\n")
    for val in range(0, len(exp_lst_arr), sample_size):
        exp_enrich_lst.append([sum(exp_lst_arr[i]) / len(exp_lst_arr[i]) for i in range(val, val + sample_size)])
    #exp_enrich = sum(exp_lst)/len(exp_lst)
    exp_enrich_lst = 0
    return exp_enrich_lst
'''


def random_loc_pe(subset_gems, chrom_size, chrom, sample_size, cov):
    np.random.seed(12345)
    exp_enrich_lst = []
    for value in subset_gems.values():
        start_idx = int(value[0][1])
        total_span = int(value[-1][2]) - start_idx
        startpos = np.random.randint(low = 0, high = chrom_size - total_span - 1, size = sample_size) - start_idx
        current_frag = []
        for pos in startpos:
            for elem in value:
                current_frag.append([chrom, pos + int(elem[1]), pos + int(elem[2])])
        exp_lst = list(cov.stats(stat = 'max', intervals = current_frag))
        ## All occurences of -1 in exp_lst:
        avg_len = len(value)
        faulty_idxs = [index for index, enr_val in enumerate(exp_lst) if enr_val == -1]
        for badidx in faulty_idxs:
            replace_idx = badidx // avg_len
            new_exps = exp_lst[replace_idx:replace_idx + avg_len]
            while -1 in new_exps:
                startpos2 = np.random.randint(low = 0, high = chrom_size - total_span - 1) - start_idx
                better_frag = []
                for elem in value:
                    better_frag.append([chrom, startpos2 + int(elem[1]), startpos2 + int(elem[2])])
                new_exps = list(cov.stats(stat = 'max', intervals = better_frag))
            for i in range(0, avg_len):
                exp_lst[replace_idx + i] = new_exps[i]
        exp_enrich_lst.append([sum(exp_lst[i:i + avg_len]) / avg_len for i in range(0, len(exp_lst), avg_len)])
        #exp_enrich = sum(exp_lst)/len(exp_lst)
    return exp_enrich_lst



#def get_raw_pval(obs_frags, obs_span, obs_fraglen, obs_f2f, sample_size, cov, bin_size, chrom_size):
def get_raw_pval(exp_enrich, obs_frags, sample_size, cov, strand_type):
    """
    Compute raw p-value for a GEM.
    Args:
       obs_frags (list of list): [chrom,start,end] for each fragment 
       obs_span (int): start of leftmost fragment to end of rightmost fragment
       obs_fraglen (list): length of each fragment
       obs_f2f (list): distance between neighboring fragments
       sample_size (int): number of pseudo-GEMs to sample
       cov: bedgraph coverage track
       bin_size (int): size of the bin used to generate cov
       chrom_size (int): size of the chromosome
    Returns:
       raw_pval (float): raw p-value computed
       obs_enrich (int): observed enrichment
    """
    # observed enrichment
#    frag_cov = []
#    for i in range(len(obs_frags)):
#        frag_cov.append(get_mean_cov(obs_frags[i][0], obs_frags[i][1], cov, bin_size))
    #frag_cov = list(cov.stats(stat = 'mean', intervals = [[obs_frags[0], int(obs_frags[1]), int(obs_frags[2])]]))
    #obs_enrich = cov.stats(stat = 'max', intervals = [[obs_frags[0], int(obs_frags[1]), int(obs_frags[2])]])[0]
    if str(strand_type) == "SE":
        obs_enrich = cov.stats(stat = 'max', intervals =  [[obs_frags[0][0], int(obs_frags[0][1]), int(obs_frags[0][2])]])[0]
    else:
        current_frag = []
        for elem in obs_frags:
            current_frag.append([elem[0], int(elem[1]), int(elem[2])])
        obs_lst = list(cov.stats(stat = 'max', intervals = current_frag))
        #new_lst = [obs_frags[:, 0].tolist(), obs_frags[:, 1].tolist(), obs_frags[:, 2].astype(int).tolist()]
        #new_lst = [obs_frags[:, 0].tolist(), obs_frags[:, 1].tolist(), obs_frags[:, 2].tolist()]
        #obs_lst = list(cov.stats(stat = 'max', intervals =  [[row[i] for row in new_lst] for i in range(len(new_lst[0]))]))
        obs_enrich = sum(obs_lst)/len(obs_lst)
        #obs_enrich = max(frag_cov)
    # expected enrichment (sampling background)
    #startpos = random_loc(chrom_size, obs_span, sample_size)
    '''
    exp_enrich = []
    for k in range(sample_size):
        pseudo = pseudo_gem(startpos[k], obs_span, chr_name)
        exp = list(cov.stats(stat = 'mean', intervals = pseudo))
        exp_enrich.append(sum(exp)/len(exp))
        #exp_enrich.append(max(exp))
    '''
    # write 1000 null (expected, sampled) values; 20191226
#    prefix_null = "_".join(prefix.split("_")[:-1]) + "_1000"
#    with open(out_directory + prefix_null + "_enrichTest_null.txt", "a") as f1:
#        f1.write(','.join(map(str, [round(x,1) for x in exp_enrich])) + '\n')
#    f1.close()
    # compute raw p-value
    #raw_pval = sum(i > obs_enrich for i in exp_enrich)/sample_size
    tot = 0
    for i in exp_enrich:
        tot += (i >= obs_enrich)
    raw_pval = tot/sample_size
    return(raw_pval, obs_enrich)

def get_adj_pval(raw_pval_list, fdr_thresh, method):
    """ 
    Adjust raw pvalues by Benjamini Hochberg multiple testing adjustment. 
    Args:
       raw_pval_list (list): list of raw p-values (ex: [0.1, 0.04, 0.1])
       fdr_thresh (float): false discovery rate (ex: 0.05)
       method (string): adjustment method (ex: 'fdr_bh')
    Returns:
       adj_pval_list (array): array of booleans and adjusted p-values (ex: [0.1, 0.1, 0.1])
    """
    adj_pval_list = multipletests(raw_pval_list, alpha = fdr_thresh, method = method)
    return(adj_pval_list)

def write_master_result(out_gem_list, out_name):
    """ 
    Write out significance test results.
    Args: 
       out_gem_list (list): list with 11 items including gem_id, frag_str, p-values, etc.
       out_name (string): output file name
    Returns:
       None
    """
    with open(out_name, 'a') as file1:
        header = ['GEM_ID', 'Start Coord', 'End Coord', 'Category', 'Obs', 'rawpval1', 'adjpval1', 'decis1', 'rawpval2', 'adjpval2', 'decis2']
        file1.write('\t'.join(map(str, header)) + '\n')
        for i in range(len(out_gem_list)):
            file1.write('\t'.join(map(str, out_gem_list[i])) + '\n')

def write_output_beds(lst, out_name):
    with open(out_name, 'a') as file:
        for line in lst:
            file.write('\t'.join(line) + '\n')

In [7]:
if __name__ == '__main__':
    #tracemalloc.start()
    total_start = time.time()
    argv = os.environ.get('NB_ARGS')
    argv = argv.split(" ")
    ### Set parameters ###
    library_name = argv[0] ## Library name of our data ##
    genome_name = argv[1] ## Name of the reference genome ##
    fdr_thresh = float(argv[2])  # should be argument; Benjamini-Hochberg FDR; p-value cutoff ##
    chr_lst = argv[3] # should be argument
    samp_size = int(argv[4]) ## Number of pseudo-GEMs ##
#    bin_size = int(argv[6])
    bg_name = argv[5] # bedgraph file name
    directory = argv[6]
    file_name = argv[7]
    strand_type = argv[8] ## Paired End or Single End (PE or SE) ##
    
    chr_lst = chr_lst.split("\n")
    args_lst = []
    for chr_name in chr_lst:
        args_lst.append((library_name, genome_name, fdr_thresh, chr_name, samp_size, bg_name, directory, file_name, strand_type))
    max_parallel = multiprocessing.cpu_count()
    split_chr_lst = []
    for i in range(0, len(args_lst), max_parallel):
        split_chr_lst.append(args_lst[i:min(i + max_parallel, len(args_lst))])
    for elem in split_chr_lst:
        print(elem)
        with multiprocessing.Pool(processes=max_parallel) as pool:
            pool.starmap(defs_v2.full_program, elem)

SyntaxError: '(' was never closed (181225878.py, line 56)